# Exercise 12 - 2025: A training with a small chemical reaction

## In this exercise we want to train a potential and test some active learning on the simple model. We thank Dr. Umberto Raucci for the collaboration. The reaction is similar to the one studied by Umberto Raucci in this [paper](https://pubs.acs.org/doi/10.1021/acs.jpclett.5c00688)



In [87]:
import numpy as np
import scipy
from scipy.special import sph_harm
from glob import glob
from ase.io import read,write
from ase import neighborlist
import matplotlib.pyplot as plt

import numpy as np
from ase.visualize import view
import matplotlib.pyplot as plt
import nglview as nv


import ase
import ase.io
import ase.lattice.cubic
import ase.md
from ase.md.nvtberendsen import NVTBerendsen
from ase.units import fs, kB
from ase.calculators.lammpsrun import LAMMPS

def view_structure(system):
    t = nv.ASEStructure(system) 
    w = nv.NGLWidget(t, gui=True)
    w.add_spacefill()
    return w


def view_trajectory(trajectory):
    t2 = nv.ASETrajectory(trajectory)
    w2 = nv.NGLWidget(t2, gui=True)
    w2.add_representation('label',label_type='atomindex',color='black')
    w2.add_representation('licorice')
    w2.add_representation('spacefill',selection=[5],color="red",radius=0.7)
    w2.add_representation('ball_and_stick')

    return w2







## First part: harvesting the "ab initio configurations" and training the potential.

 ### Assigment 1.
 Go to the subdirectory <code>s0/00_iteration/newmodel</code> and train the potential with the command
 
 <code>bash train_1.sh  </code> . Add an "&" if you want to see the files during the run. 
 
 Plot the error in the training along the training and show the decay. Comment on it. 
 Even if the training is not over, stop the training, "touch" the file <code>tag_0_finished</code> and rerun **train_1.sh** to create the frozen model and the compressed one at this point. You can always restart later.

 ### Assigment 2.
 Go to the subdirectory <code>s0/00_iteration/dyn_bias/b120_meta</code> and run the metadynamics run with 

<code>bash run_meta.sh >& meta.out & </code>
 
 First, observe the deviation among the potentials. See cell below. What do you observe?
 What happens if you repeat the metadynamics adding your "not converged" new_model and keeping also 1, 2, 3 that are perfectly converged?


In [ ]:
#
# Reading the deviations among the 4 potential (for later active learning)
#

import matplotlib.pyplot as plt

# Read data from the text file
column_4_values = []
with open("s0/00_iteration/dyn_bias/b120_meta/model_devi.out", "r") as file:
    for line in file:
        if line.strip() and not line.startswith("#"):  # Ignore empty lines and comments
            parts = line.split()
            if len(parts) >= 4:  # Ensure there are at least 4 columns
                column_4_values.append(float(parts[4]))  # Column indexing starts at 0

# Plot histogram
plt.figure(figsize=(8,5))
plt.hist(column_4_values, bins=200, edgecolor='black', alpha=0.7)

plt.xlabel("Force Deviation among 4 potentials")
plt.ylabel("Frequency")
plt.title("Max dev forces (eV/A/atom)")
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()


## Assigment 3.

Monitor the COLVAR behavior over time (next cell) as well as the HILLS over time (next cell afterwards). What is the correlation between these two plots? Comment the correlation between the CV value and the trajectory. The colvar is defined in the <code>plumed.dat</code> file. 

After a long simulation time, your free energy should be quite converged. The units are kj/mol. What can you conclude about the proton transfer process?


In [ ]:
#
# meta
#

#
# Plot the dynamics
#

import matplotlib.pyplot as plt
import numpy as np

# Load data from text file (assuming columns are separated by spaces or tabs)
!cp s0/00_iteration/dyn_bias/b120_meta/COLVAR C2
data = np.loadtxt('C2', usecols=(0,3))
data = data [:-1]

print (len(data))

# Extract first and second columns
x = data[:, 0]  # First column
y = data[:, 1]  # Second column

# Plot
plt.plot(x, y, marker='o', linestyle='-')
plt.xlabel('time')
plt.ylabel('CV')
plt.title('CV in time')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load HILLS file (ignoring comment lines)

df = pd.read_csv('s0/00_iteration/dyn_bias/b120_meta/HILLS', sep='\s+', comment='#', header=None)
df.columns = ["time", "cv", "sigma_cv", "height", "biasf"]

# Define a grid over the CV space
cv_min, cv_max = df["cv"].min(), df["cv"].max()
cv_grid = np.linspace(cv_min, cv_max, 1000)  # High resolution grid

# Initialize cumulative bias array
cumulative_bias = np.zeros((len(df), len(cv_grid)))

# Compute cumulative bias over time
for i in range(len(df)):
    h, c, sigma = df.loc[i, ["height", "cv", "sigma_cv"]]
    cumulative_bias[i] = cumulative_bias[i - 1] + h * np.exp(-((cv_grid - c)**2) / (2 * sigma**2))

# Plot successive cumulative bias landscapes
plt.figure(figsize=(8, 6))
for i in range(0, len(df), max(1, len(df) // 10)):  # Plot 10 evenly spaced curves
    plt.plot(cv_grid, cumulative_bias[i], label=f'Time {df.loc[i, "time"]:.1f}')

plt.xlabel("Collective Variable (CV)")
plt.ylabel("Accumulated Bias Potential")
plt.title("Cumulative Sum of HILLS Over Time")
plt.legend(loc="upper left", fontsize="small")
plt.show()


In [ ]:
#
# Reading the trajectory
# 

mytraj = read ("s0/00_iteration/dyn_bias/b120_meta/dump.xyz",":")
view_trajectory (mytraj)

# Assignment 4: do the same with OPES

Move to the directory <code>s0/00_iteration/dyn_bias/b120_opes</code> and execute

<code> bash run_opes.sh > opes.out & </code>

Monitor COLVAR, the trajectory, the histogram of the COLVAR, the bias in time, and the FES obtained with a script. 
Comment on all this and compare with the META example as well as the considerations in the [PLUMED WEBSITE](https://www.plumed.org/doc-v2.9/user-doc/html/opes-metad.html).

Do you observe many transitions? What happens in the system? What does the FES tell us?


In [ ]:
#
# Plot the dynamics
#

import matplotlib.pyplot as plt
import numpy as np

# Load data from text file (assuming columns are separated by spaces or tabs)
!cp s0/00_iteration/dyn_bias/b120_opes/COLVAR C2
data = np.loadtxt('C2',usecols=(0,3))

print (len(data))

# Extract first and second columns
x = data[:, 0]  # First column
y = data[:, 1]  # Second column

# Plot
plt.plot(x, y, marker='o', linestyle='-')
plt.xlabel('Snapshot')
plt.ylabel('COLVAR')
plt.title('CH_2-CH_1')
plt.show()


In [ ]:
#
# Reading the trajectory
# 

mytraj = read ("s0/00_iteration/dyn_bias/b120_opes/dump.xyz",":")

In [ ]:
view_trajectory(mytraj)

In [ ]:
#
# Plotting the BIAS in time
#


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load COLVAR file (ignoring comment lines)
df = pd.read_csv("s0/00_iteration/dyn_bias/b120_opes/COLVAR", sep='\s+', comment='#', header=None)

# Extract relevant columns
x_values = df.iloc[:, 3]   # Fourth column
y_values = -df.iloc[:, 4]  # Fifth column (changing sign)
time_values = df.iloc[:, 0]  # First column for coloring

# Create scatter plot with color coding by time
plt.figure(figsize=(8, 6))
scatter = plt.scatter(x_values, y_values, c=time_values, cmap='viridis', edgecolor='k')
plt.colorbar(scatter, label="Time")
plt.xlabel("Column 4 (CV)")
plt.ylabel("Negated Column 5 (Bias)")
plt.title("Bias Visualization")
plt.show()


In [ ]:
# Histogram of the colvar

#
# Reading the deviations
#

import matplotlib.pyplot as plt

# Read data from the text file
column_4_values = []
with open("s0/00_iteration/dyn_bias/b120_opes/COLVAR", "r") as file:
    for line in file:
        if line.strip() and not line.startswith("#"):  # Ignore empty lines and comments
            parts = line.split()
            if len(parts) >= 4:  # Ensure there are at least 4 columns
                column_4_values.append(float(parts[3]))  # Column indexing starts at 0

# Plot histogram
plt.figure(figsize=(8,5))
plt.hist(column_4_values, bins=200, edgecolor='black', alpha=0.7)

plt.xlabel("Colvar (angstrom)")
plt.ylabel("Frequency")
plt.title("Histogram of Colvar")
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()


In [ ]:
#
#Extract FES from the BIAS by applying appropriate weigths

#
# Plot the dynamics
#

import matplotlib.pyplot as plt
import numpy as np

# Load data from text file (assuming columns are separated by spaces or tabs)
!cat s0/00_iteration/dyn_bias/b120_opes/COLVAR  | head -n -1 > C2 
!pwd
!cp s0/00_iteration/dyn_bias/b120_opes/FES_from_Reweighting.py .
!python ./FES_from_Reweighting.py --sigma 0.08 --temp 300 --colvar C2  --cv 4 
data = np.loadtxt('fes-rew.dat')

print (len(data))

# Extract first and second columns
x = data[:, 0]  # First column
y = data[:, 1]  # Second column

# Plot
plt.plot(x, y, marker='o', linestyle='-')
plt.xlabel('CV')
plt.ylabel('Free energy')
plt.title('FES from OPES')
plt.show()